In [3]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
from jupyter_dash import JupyterDash  # Import JupyterDash instead of Dash
import dash_bootstrap_components as dbc
import pandas as pd
import plotly.express as px
from dash import html, dcc, callback, Output, Input

# Load and process the data
df = pd.read_csv('data1.csv')
df.columns = df.columns.str.strip()

# Calculate averages
avg_math_score = round(df.get('MathScore', pd.Series()).mean(), 2)
avg_reading_score = round(df.get('ReadingScore', pd.Series()).mean(), 2)
avg_writing_score = round(df.get('WritingScore', pd.Series()).mean(), 2)

# Initialize app
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])  # Use JupyterDash

# Create reusable card component
def get_card_component(title, data):
    component = dbc.Col(
                    dbc.Card(
                        dbc.CardBody([
                            html.H4(title),
                            html.H4(data)
                        ]), 
                        color="dark", 
                        outline=True,
                        className = 'text-dark',
                        style={'textAlign': 'center', 'margin-bottom': '20px'}
                    ),
                )
    return component

# Layout
app.layout = html.Div([
    html.H1(children='Student Exam Scores', style={'textAlign':'center', 'padding-bottom': '20px'}),
    
    dbc.Row([
        get_card_component('Total Students', '{:,}'.format(len(df.index))),
        get_card_component('Avg Math Score', str(avg_math_score)),
        get_card_component('Avg Writing Score', str(avg_writing_score)),
        get_card_component('Avg Reading Score', str(avg_reading_score)),
    ]),

    # Score Distribution Section
    dbc.Row(
        dbc.Col([
            html.H4("Score Distribution"),
            html.Div(
                dbc.RadioItems(
                    id="score-distribution-radios",
                    className="btn-group",
                    inputClassName="btn-check",
                    labelClassName="btn btn-outline-dark",
                    options=[
                        {'label': 'Math Score', 'value': 'MathScore'},
                        {'label': 'Writing Score', 'value': 'WritingScore'},
                        {'label': 'Reading Score', 'value': 'ReadingScore'},
                    ],
                    value='MathScore',
                ),
                className="radio-group",
                style={'margin-top': '20px'}
            ),
            dcc.Graph(figure={}, id="score-distribution-histogram")
        ])
    ),

    # Categorical Data Exploration Section
    dbc.Row([
        html.H4("Explore Categorical Data"),
        dbc.Col(
            dcc.RadioItems(
                id="student-categorical-radios",
                className="btn-group",
                inputClassName="btn-check",
                labelClassName="btn btn-outline-dark",
                options=[
                    {'label': 'Gender', 'value': 'Gender'},
                    {'label': 'Ethnic Group', 'value': 'EthnicGroup'},
                    {'label': 'Parent Education', 'value': 'ParentEduc'},
                    {'label': 'Parent Marital Status', 'value': 'ParentMaritalStatus'},
                    {'label': 'Test Preparation', 'value': 'TestPrep'},
                    {'label': 'Weekly Study Hours', 'value': 'WklyStudyHours'},
                ],
                value='Gender',
            ),
            className="radio-group",
            style={'margin-top': '20px'}
        ),
        dcc.Graph(figure={}, id="student-categorical-summary"),
        dbc.Row([
            dbc.Col(dcc.Graph(figure={}, id="student-categorical-math-score-distribution")),
            dbc.Col(dcc.Graph(figure={}, id="student-categorical-writing-score-distribution")),
            dbc.Col(dcc.Graph(figure={}, id="student-categorical-reading-score-distribution")),
        ]),
    ])
], style={"margin": "50px 50px 50px 50px"})

@callback(
    Output("student-categorical-summary", "figure"),
    Output("student-categorical-math-score-distribution", "figure"),
    Output("student-categorical-writing-score-distribution", "figure"),
    Output("student-categorical-reading-score-distribution", "figure"),
    Input("student-categorical-radios", "value")
)
def update_all_figures(value='Gender'):
    # Define the color sequence inside the function
    color_discrete_sequence = ['#0a9396','#94d2bd','#e9d8a6','#ee9b00', '#ca6702', '#bb3e03', '#ae2012']
    
    # Grouped data for summary figure
    grouped_df = df[value].value_counts().reset_index()
    grouped_df.columns = [value, 'Count']
    
    # Create summary figure
    summary_figure = px.bar(
        grouped_df, 
        x=value, 
        y='Count', 
        color=value, 
        color_discrete_sequence=color_discrete_sequence, 
        title='Summary of ' + value
    )

    # Create score distribution figures
    math_score_figure = px.box(df, x=value, y="MathScore", color_discrete_sequence=['#0a9396'], title='Math Score Distribution')
    writing_score_figure = px.box(df, x=value, y="WritingScore", color_discrete_sequence=['#ee9b00'], title='Writing Score Distribution')
    reading_score_figure = px.box(df, x=value, y="ReadingScore", color_discrete_sequence=['#bb3e03'], title='Reading Score Distribution')

    return summary_figure, math_score_figure, writing_score_figure, reading_score_figure
    # Callback to update histogram based on score selection
@callback(
    Output("score-distribution-histogram", "figure"), 
    Input("score-distribution-radios", "value")
)
def update_histogram(value):
    figure = px.histogram(df, x=value, color_discrete_sequence=['#0a9396'], title=f'{value} Distribution')
    return figure

# Run the app
if __name__ == '__main__':
    app.run(debug=True, port=8051)
